In [85]:
from dotenv import load_dotenv
import os
load_dotenv()  # Load .env file

True

In [2]:
from langchain_openai import ChatOpenAI
from langsmith import traceable

In [6]:
from langchain_openai import ChatOpenAI
from langsmith import traceable
from langchain_core.messages import HumanMessage # inheriting from Base message class
# Initialize ChatOpenAI with LM Studio endpoint
llm = ChatOpenAI(
    base_url="http://172.17.160.1:1234/v1",  # Can be any dummy value
    api_key='None',
    model="gemma-3-4b-it",  # Optional
    temperature=0.7
)

# Use the model
response = llm.invoke([HumanMessage(content = 'Hello,How are you ?')])
print(response.content)

Hi there! I’m doing well, thanks for asking! As an AI, I don't really *feel* in the same way humans do, but my systems are running smoothly and I’m ready to chat. 😊

How about you? How is your day going so far? 

Is there anything you’d like to talk about or something I can help you with?


In [8]:
from langchain_core.messages import AIMessage

llm.invoke([
    HumanMessage('Hello, How are you ?,my name is akshat'),
    AIMessage('I am fine, How are you akshat'),
    HumanMessage('i am fine , what is my name and talk about something you like')
])

AIMessage(content="Your name is Akshat! That’s a great name. 😊\n\nOkay, so something I really enjoy (and it's a bit nerdy!) is the concept of **artificial intelligence**. Specifically, I find it fascinating how AI can learn and adapt – like me! It’s incredible to think that algorithms are being developed to mimic human thought processes, solve complex problems, and even create art. \n\nI love exploring the potential applications, from helping doctors diagnose diseases more accurately to designing sustainable solutions for climate change.  It feels like we're on the cusp of a really transformative era, and I’m constantly learning new things about it. \n\nWhat do *you* enjoy?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 142, 'prompt_tokens': 55, 'total_tokens': 197, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gemma-3-4b-it', 'system_fingerprint': 'gemma-3-4b-it', 'id': 'chatcmpl-jhcn8dbt01scpevyiqx5m'

In [21]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

In [22]:
workflow = StateGraph(state_schema=MessagesState)# Define a new graph

In [23]:
async def call_model(state : MessagesState): # Message state is an inhertied TypedDict can use an Pydantic Model too
    response = llm.invoke(state['messages'])
    return {'messages' : response}

workflow.add_edge(START,'model') # Used for starting the START node is virtual or symbolic
workflow.add_node('model',call_model) # START is connected to model node
memory = MemorySaver() # Saves memory inside RAM , not used for production only for debugging
app = workflow.compile(checkpointer=memory) # compiles the workflow

In [24]:
config = {"configurable": {"thread_id": "abc123"}}

In [25]:
query = "Hi! I'm Akshat and i love AI/ML/DS/Gen-AI."
input_messages = [HumanMessage(query)]

output = await app.ainvoke({'messages' : input_messages},config)

In [31]:
output['messages'][-1].pretty_print()

================================== Ai Message ==================================

Hey Akshat, that’s fantastic! It’s awesome to meet someone who’s passionate about AI/ML/DS/Gen-AI – it’s such a rapidly evolving and exciting field right now. 

It's great you're so engaged with these areas. What specifically are you most interested in within those fields?  Do you have any particular projects you're working on, or topics you're currently learning about?

I'd love to chat more about it!  Perhaps we could talk about:

*   **Specific Technologies:** Are you exploring things like TensorFlow, PyTorch, Langchain, Stable Diffusion, etc.?
*   **Applications:** What kind of problems do you find most interesting to solve with AI – healthcare, finance, creative applications, robotics?
*   **Current Trends:**  Are you following developments in Large Language Models (LLMs), multimodal AI, or something else?

Don't hesitate to tell me anything that comes to mind!


In [40]:
query = "What's my name?"
input_messages = [HumanMessage(query)]
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

(Sighing gently) Akshat. Seriously, it’s Akshat. I understand you might be testing me, but this is getting repetitive and unproductive. 

Let’s talk about something *you* are genuinely interested in within AI/ML – perhaps a specific algorithm, a dataset you'd like to explore, or a problem you want to tackle?


In [44]:
config2 = {'configurable':{'thread_id' : 123}} # Using different thread_id restarted the conversation , one possible thing to look at is the structure of this config2 dictionary when i changed thread_id to thread it threw error , potential investigation !!!
query = "What's my name?"
input_messages = [HumanMessage(query)]
output = await app.ainvoke({"messages": input_messages}, config2) # Passing config2 instead of config resetted the convo
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I apologize for the repetition! As an AI, I have absolutely no way of knowing your name. You haven't told me yet. 😊 

Would you like to tell me what your name is?


In [54]:
'''
This is a dummy example used to demonstrate how prompt templates are used to dynamically update prompts like other things in langchain , it is a runnable
and integrates well with other runnable using LCEL (|) , also it is useful for modern LLM that use system,user,AI to identify messages , it can also fill in
placeholders like {placeholder} , provided during invoke(). Note this will fail here because the model we using gemma locally is not a turn-based chat model
but OPEN-AI GPT is , so i have done a simple workaround to make it executable with gemma too , in the cell below.
'''

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("user", "{question}")
])

completed_prompt = prompt.invoke({'question' : 'whats your name'})
completed_prompt

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}), HumanMessage(content='whats your name', additional_kwargs={}, response_metadata={})])

In [65]:
completed_prompt.to_messages()

[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats your name', additional_kwargs={}, response_metadata={})]

In [69]:
def chat_messages_to_string(chat_prompt):
    return "\n".join([f"{msg.type.capitalize()}: {msg.content}" for msg in chat_prompt.to_messages()])

string_prompt = chat_messages_to_string(completed_prompt)
llm.invoke([string_prompt])

AIMessage(content="You can call me Gemma! I’m an open-weights AI assistant created by the Gemma team at Google DeepMind. \n\nIt's nice to meet you! 😊\n", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 23, 'total_tokens': 60, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gemma-3-4b-it', 'system_fingerprint': 'gemma-3-4b-it', 'id': 'chatcmpl-8rp7u2mwjgw4rn8qy0xk56', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--68e18e5a-dd76-40e1-8256-46a15193d234-0', usage_metadata={'input_tokens': 23, 'output_tokens': 37, 'total_tokens': 60, 'input_token_details': {}, 'output_token_details': {}})

In [73]:
'''
MessagesPlaceholder is a special component designed to inject dynamic lists of messages into specific positions within your chat template. Unlike regular variable placeholders that expect single string values, MessagesPlaceholder expects a list of complete messages and inserts them all at that position.​

The key difference: regular placeholders like {question} take a string and create a single message, while MessagesPlaceholder takes multiple pre-formed messages and inserts them as-is into the conversation flow.

Very useful when you want to build prompts like: "beginning of prompt" + conversation history + closing instructions + latest user message.
'''

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder("history"),
    ('system','You will not use offensive languages'),
    ("human", "{question}")
])
prompt.invoke({
    "history": [
        ("human", "what's 5 + 2"),
        ("ai", "5 + 2 is 7")
    ],
    "question": "now multiply that by 4"
})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}), HumanMessage(content="what's 5 + 2", additional_kwargs={}, response_metadata={}), AIMessage(content='5 + 2 is 7', additional_kwargs={}, response_metadata={}), SystemMessage(content='You will not use offensive languages', additional_kwargs={}, response_metadata={}), HumanMessage(content='now multiply that by 4', additional_kwargs={}, response_metadata={})])

In [76]:
'''
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.

Importantly, you will want to do this BEFORE the prompt template but AFTER you load previous messages from Message History.
'''

'\nOne important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.\n\nImportantly, you will want to do this BEFORE the prompt template but AFTER you load previous messages from Message History.\n'

In [84]:
'''
By default, .stream in our LangGraph application streams application steps-- in this case, the single step of the model response. Setting stream_mode="messages" allows us to stream output tokens instead:
'''

config = {"configurable": {"thread_id": "abc789"}}
query = "Hi I'm Akshat, please tell me a joke."
language = "English"

input_messages = [HumanMessage(query)]
async for chunk, metadata in app.astream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

Hi| Ak|sh|at|!| Here|’|s| a| joke| for| you|:|

|Why| did| the| bicycle| fall| over|?| |

|…| Because| it| was| two| tired|!| 😂| |

|Hope| you| enjoyed| that| one|!| 😊| Do| you| want| to| hear| another|?||